# Week 7 Exercise: Fine-Tuned LLaMA 3.1 Price Prediction

Evaluating fine-tuned LLaMA 3.1 8B model for product price estimation


In [ ]:
# Install required libraries for model inference
%pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
%pip install -q --upgrade requests==2.32.3 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib wandb

In [ ]:
# Imports
import os
import re
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed
from datasets import load_dataset
from peft import PeftModel
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

%matplotlib inline
set_seed(42)


In [ ]:
# Configuration
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
HF_USER = "ed-donner"
PROJECT_NAME = "pricer"
RUN_NAME = "2024-09-13_13.04.39"
REVISION = "e8d637df551603dc86cd7a1598a8f44af4d7ae36"

FINETUNED_MODEL = f"{HF_USER}/{PROJECT_NAME}-{RUN_NAME}"
DATASET_NAME = f"{HF_USER}/pricer-data"
QUANT_4_BIT = False


In [ ]:
# Login to HuggingFace
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)


In [ ]:
# Load dataset with token
hf_token = userdata.get('HF_TOKEN')
dataset = load_dataset(DATASET_NAME, token=hf_token)
train = dataset['train']
test = dataset['test']

print(f"Train: {len(train)} samples")
print(f"Test: {len(test)} samples")


In [ ]:
# Configure quantization
if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )
else:
    quant_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16
    )


In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load base model with quantization
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

# Load fine-tuned weights
if REVISION:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL, revision=REVISION)
else:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

print(f"Model loaded - Memory: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")

In [ ]:
# Helper functions
import math

def extract_price(text):
    if "Price is $" in text:
        contents = text.split("Price is $")[1]
        contents = contents.replace(',', '')
        match = re.search(r"[-+]?\d*\.\d+|\d+", contents)
        return float(match.group()) if match else 0.0
    return 0.0

def predict_price(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(fine_tuned_model.device)
    attention_mask = torch.ones(inputs.shape, device=fine_tuned_model.device)

    with torch.no_grad():
        outputs = fine_tuned_model.generate(
            inputs,
            attention_mask=attention_mask,
            max_new_tokens=10,
            num_return_sequences=1
        )

    response = tokenizer.decode(outputs[0])
    return extract_price(response)

# Evaluation class
GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red": RED, "orange": YELLOW, "green": GREEN}

In [ ]:
# Improved prediction with weighted top-K
import torch.nn.functional as F

def predict_price_weighted(prompt, top_k=6):
    """
    Weighted top-K prediction for more robust price estimates.
    Takes top K most likely tokens and computes weighted average.
    """
    set_seed(42)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(fine_tuned_model.device)
    attention_mask = torch.ones(inputs.shape, device=fine_tuned_model.device)

    with torch.no_grad():
        outputs = fine_tuned_model(inputs, attention_mask=attention_mask)
        next_token_logits = outputs.logits[:, -1, :].to('cpu')

    # Get probabilities and top K tokens
    next_token_probs = F.softmax(next_token_logits, dim=-1)
    top_prob, top_token_id = next_token_probs.topk(top_k)

    # Extract prices from top tokens
    prices, weights = [], []
    for i in range(top_k):
        predicted_token = tokenizer.decode(top_token_id[0][i])
        probability = top_prob[0][i]
        try:
            result = float(predicted_token)
        except ValueError:
            result = 0.0
        if result > 0:
            prices.append(result)
            weights.append(probability.item())

    if not prices:
        return 0.0

    # Compute weighted average
    total = sum(weights)
    weighted_prices = [price * weight / total for price, weight in zip(prices, weights)]
    return sum(weighted_prices)

print("Weighted top-K prediction ready!")

In [ ]:
# Evaluation framework
class Tester:
    def __init__(self, predictor, data, title=None, size=250):
        self.predictor = predictor
        self.data = data
        self.title = title or predictor.__name__.replace("_", " ").title()
        self.size = size
        self.guesses = []
        self.truths = []
        self.errors = []
        self.sles = []
        self.colors = []

    def color_for(self, error, truth):
        if error<40 or error/truth < 0.2:
            return "green"
        elif error<80 or error/truth < 0.4:
            return "orange"
        else:
            return "red"

    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint["text"])
        truth = datapoint["price"]
        error = abs(guess - truth)
        log_error = math.log(truth+1) - math.log(guess+1)
        sle = log_error ** 2
        color = self.color_for(error, truth)
        title = datapoint["text"].split("\n\n")[1][:20] + "..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color)
        print(f"{COLOR_MAP[color]}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        max_val = max(max(self.truths), max(self.guesses))
        plt.plot([0, max_val], [0, max_val], color='deepskyblue', lw=2, alpha=0.6)
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.xlim(0, max_val)
        plt.ylim(0, max_val)
        plt.title(title)
        plt.show()

    def report(self):
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = sum(1 for color in self.colors if color=="green")
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={hits/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()

    @classmethod
    def test(cls, function, data):
        cls(function, data).run()

In [ ]:
# Run evaluation
Tester.test(predict_price_weighted, test)
